In [ ]:
import os 
%pwd

In [ ]:
%pwd

In [ ]:
!dir

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
# Entity 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file: Path
    unzip_dir : Path


In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config


In [ ]:
#!pip install gdown#

In [ ]:
import os 
import urllib.request as request
import gdown
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):

            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename= self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def google_drive_down(self):
        
        if not os.path.exists(self.config.local_data_file):

            prefix = 'https://drive.google.com/uc?/export=download&id='

            url = self.config.source_URL
            output = self.config.local_data_file
            
            file_id = url.split('/')[-2]

            gdown.download(prefix+file_id, output)

            logger.info(f"{output} download! with following info: \n")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

        


    def extract_zip_file(self):

        """
        zip_file_path:str 
        Extracts the zip file into the data directory
        Function returns None 
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [ ]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.google_drive_down()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e